In [1]:
%run ../../Utils/yp_utils.py

# Initial setup

In [2]:
paper_pmid = 24040048
paper_name = 'svensson_samson_2013' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [21]:
original_data = pd.read_excel('raw_data/logratios(treatedcontrol)-3807strains.xlsx', sheet_name='Sheet 1')

In [22]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 3807 x 18


In [23]:
original_data.head()

,ID,NAME,MMS10,BCNU10,DMSO10,ENU10,MC10,MEN10,HYQ10,MNU10,MMS20,BCNU20,DMSO20,ENU20,MC20,MEN20,HYQ20,MNU20
0,YAL004W,YAL004W,-0.053899,-0.163378,0.000000,0.000000,-0.024427,0.000000,-0.848196,-0.431291,-0.034744,-0.455707,0.000000,-0.011639,0.000000,-0.206491,-0.508631,-0.283234
1,YAL005C,SSA1,-0.091459,-0.401777,0.000000,0.000000,-0.000013,-0.037995,-0.239528,-0.206127,0.000000,-0.303430,0.000000,0.000000,0.000000,-0.063501,-0.128405,-0.112967
2,YAL008W,FUN14,-0.088300,-0.227518,0.000000,0.000000,-0.011679,-0.124014,0.000000,0.000000,-0.040763,-0.974317,-0.151552,0.000000,-0.547147,-0.136170,-0.098280,0.000000
3,YAL011W,SWC1,-0.214161,-0.418467,-2.271941,0.000000,-1.588559,-1.111195,-0.584643,-0.055866,-0.869161,-0.717548,-1.598751,0.000000,-3.071141,-0.834280,-0.452792,0.000000
4,YAL012W,CYS3,0.000000,-1.267253,0.000000,-0.471826,0.000000,0.000000,0.000000,-0.088202,-0.201033,-0.187599,0.000000,-0.232470,0.000000,0.000000,-0.163552,-0.121886


In [24]:
original_data['orf'] = original_data['ID'].astype(str)

In [25]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [26]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [27]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ID, NAME, MMS10, BCNU10, DMSO10, ENU10, MC10, MEN10, HYQ10, MNU10, MMS20, BCNU20, DMSO20, ENU20, MC20, MEN20, HYQ20, MNU20, orf]
Index: []


In [28]:
original_data.set_index('orf', inplace=True)

In [29]:
original_data.drop(columns=['NAME'], inplace=True)

In [30]:
original_data = original_data.groupby(original_data.index).mean()

In [31]:
original_data.shape

(3790, 16)

# Prepare the final dataset

In [32]:
data = original_data.copy()

In [33]:
dataset_ids = [5217, 5221, 5220, 5219, 5222, 5223, 5224, 5218, 5225, 5229, 5228, 5227, 5230, 5231, 5232, 5226]
datasets = datasets.reindex(index=dataset_ids)

In [34]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [35]:
data.head()

dataset_id,5217,5221,5220,5219,5222,5223,5224,5218,5225,5229,5228,5227,5230,5231,5232,5226
data_type,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value
orf,,,,,,,,,,,,,,,,
YAL004W,-0.053899,-0.163378,0.000000,0.000000,-0.024427,0.000000,-0.848196,-0.431291,-0.034744,-0.455707,0.000000,-0.011639,0.000000,-0.206491,-0.508631,-0.283234
YAL005C,-0.091459,-0.401777,0.000000,0.000000,-0.000013,-0.037995,-0.239528,-0.206127,0.000000,-0.303430,0.000000,0.000000,0.000000,-0.063501,-0.128405,-0.112967
YAL008W,-0.088300,-0.227518,0.000000,0.000000,-0.011679,-0.124014,0.000000,0.000000,-0.040763,-0.974317,-0.151552,0.000000,-0.547147,-0.136170,-0.098280,0.000000
YAL011W,-0.214161,-0.418467,-2.271941,0.000000,-1.588559,-1.111195,-0.584643,-0.055866,-0.869161,-0.717548,-1.598751,0.000000,-3.071141,-0.834280,-0.452792,0.000000
YAL012W,0.000000,-1.267253,0.000000,-0.471826,0.000000,0.000000,0.000000,-0.088202,-0.201033,-0.187599,0.000000,-0.232470,0.000000,0.000000,-0.163552,-0.121886


## Subset to the genes currently in SGD

In [36]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 19


In [37]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,5217,5221,5220,5219,5222,5223,5224,5218,5225,5229,5228,5227,5230,5231,5232,5226
,data_type,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value,value
gene_id,orf,,,,,,,,,,,,,,,,
1863,YAL004W,-0.053899,-0.163378,0.000000,0.000000,-0.024427,0.000000,-0.848196,-0.431291,-0.034744,-0.455707,0.000000,-0.011639,0.000000,-0.206491,-0.508631,-0.283234
4,YAL005C,-0.091459,-0.401777,0.000000,0.000000,-0.000013,-0.037995,-0.239528,-0.206127,0.000000,-0.303430,0.000000,0.000000,0.000000,-0.063501,-0.128405,-0.112967
6,YAL008W,-0.088300,-0.227518,0.000000,0.000000,-0.011679,-0.124014,0.000000,0.000000,-0.040763,-0.974317,-0.151552,0.000000,-0.547147,-0.136170,-0.098280,0.000000
9,YAL011W,-0.214161,-0.418467,-2.271941,0.000000,-1.588559,-1.111195,-0.584643,-0.055866,-0.869161,-0.717548,-1.598751,0.000000,-3.071141,-0.834280,-0.452792,0.000000
10,YAL012W,0.000000,-1.267253,0.000000,-0.471826,0.000000,0.000000,0.000000,-0.088202,-0.201033,-0.187599,0.000000,-0.232470,0.000000,0.000000,-0.163552,-0.121886


# Normalize

In [38]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [39]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [40]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,5217,5221,5220,5219,5222,5223,5224,5218,5225,5229,...,5224,5218,5225,5229,5228,5227,5230,5231,5232,5226
,data_type,value,value,value,value,value,value,value,value,value,value,...,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez,valuez
gene_id,orf,,,,,,,,,,,,,,,,,,,,,
1863,YAL004W,-0.053899,-0.163378,0.000000,0.000000,-0.024427,0.000000,-0.848196,-0.431291,-0.034744,-0.455707,...,-0.660141,-1.146425,0.049238,-0.785453,0.096358,-0.030132,0.000000,-0.182303,-0.390673,-0.830167
4,YAL005C,-0.091459,-0.401777,0.000000,0.000000,-0.000013,-0.037995,-0.239528,-0.206127,0.000000,-0.303430,...,-0.136538,-0.547912,0.098804,-0.522989,0.096358,0.000000,0.000000,0.025104,-0.043463,-0.331108
6,YAL008W,-0.088300,-0.227518,0.000000,0.000000,-0.011679,-0.124014,0.000000,0.000000,-0.040763,-0.974317,...,0.069515,0.000000,0.040652,-1.679324,-0.106056,0.000000,-0.635062,-0.080302,-0.015954,0.000000
9,YAL011W,-0.214161,-0.418467,-2.271941,0.000000,-1.588559,-1.111195,-0.584643,-0.055866,-0.869161,-0.717548,...,-0.433421,-0.148500,-1.141129,-1.236759,-2.038943,0.000000,-3.564607,-1.092912,-0.339683,0.000000
10,YAL012W,0.000000,-1.267253,0.000000,-0.471826,0.000000,0.000000,0.000000,-0.088202,-0.201033,-0.187599,...,0.069515,-0.234453,-0.187987,-0.323344,0.096358,-0.601837,0.000000,0.117212,-0.075559,-0.357250


# Print out

In [41]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [42]:
from IO.save_data_to_db3 import *

In [43]:
save_data_to_db(data_all, paper_pmid)

Deleting all datasets for PMID 24040048...


  0%|          | 0/16 [00:00<?, ?it/s]

Inserting the new data...


100%|██████████| 16/16 [01:24<00:00,  5.31s/it]

Updating the data_modified_on field...
